<h1 style="text-align: center;">Script for creating and analyzing longitudinal data from the Tec21 academic model Cohort 2019 and 2020</h1>

#### School: Engineering and Science
#### Period: AD2019-FJ2022

#### Table of contents                                                                                                                
                                            
- [1. Datasets creation](#1.-Datasets-creation)                                                                               
    - [1.1 Data subset extraction](#1.1-Data-subset-extraction)                                                                     
    - [1.2 Statistical description of the selected variables](#1.2-Statistical-description-of-the-selected-variables)               
    - [1.3 Graphical description of the selected variables](#1.3-Graphical-description-of-the-selected-variables)                   

In [ ]:
# Importing needed libraries
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import plotly.express as px
from ipywidgets import widgets
from statsmodels.formula.api import ols
from statsmodels.stats.multitest import multipletests
import plotly.express as px
warnings.filterwarnings('ignore')
path_files = r'EICData'

In [ ]:
# Load the dataset
full_df = pd.read_csv(f'EICData\Transformed_full_df_with_studentid_v1.0.csv', index_col='Unnamed: 0')
print(f'Dataframe loaded! \n The tidy dataframe has {len(full_df.index)} rows and {len(full_df.columns)} columns.')

In [ ]:
# Temporary code for checking dataset

len(full_df[(full_df['program.major_id'] == 'IBQ')
        & (full_df['competence.type'] == 'Disciplinary')]['student.id'].unique())

### 1.1-Data-subset-extraction

In [ ]:
# Select the Cohort
atts = full_df['student.cohort.id'].unique().tolist()
btn_atts = widgets.Button(description='Select')
mult_choice = widgets.SelectMultiple(
    options=atts,
    description='Select cohort',
    disabled=False
)
display(mult_choice)
display(btn_atts)
def btn_select_att(btn_atts):
    global cohort
    cohort = list(mult_choice.value)
btn_atts.on_click(btn_select_att)

#### Select programs

In [ ]:
# Select the programs
programs = full_df['program.major_id'].unique().tolist()
atts = full_df['program.major_id'].unique().tolist()
btn_atts = widgets.Button(description='Select')
mult_choice = widgets.SelectMultiple(
    options=atts,
    description='Select programs',
    disabled=False
)
display(mult_choice)
display(btn_atts)
def btn_select_att(btn_atts):
    global programs
    programs = list(mult_choice.value)
btn_atts.on_click(btn_select_att)

#### Select region

In [ ]:
# Select the region
regions = full_df['campus.region_name'].unique().tolist()
atts = full_df['campus.region_name'].unique().tolist()
btn_atts = widgets.Button(description='Select')
mult_choice = widgets.SelectMultiple(
    options=atts,
    description='Select regions',
    disabled=False
)
display(mult_choice)
display(btn_atts)
def btn_select_att(btn_atts):
    global regions
    regions = list(mult_choice.value)
btn_atts.on_click(btn_select_att)

#### Select sex

In [ ]:
# Select the sex
sex = full_df['student.isWoman'].unique().tolist()
atts = full_df['student.isWoman'].unique().tolist()
btn_atts = widgets.Button(description='Select')
mult_choice = widgets.SelectMultiple(
    options=atts,
    description='Select sex',
    disabled=False
)
display(mult_choice)
display(btn_atts)
def btn_select_att(btn_atts):
    global sex
    sex = list(mult_choice.value)
btn_atts.on_click(btn_select_att)

#### Select grouping var

In [ ]:
# Select the region
atts = ['campus.region_name', 'student.cohort.id','program.major_id', 'student.isWoman']
btn_atts = widgets.Button(description='Select')
cb_list = widgets.Dropdown(
    options=atts,
    description='Select grouping var',
    disabled=False,
)
display(cb_list)
display(btn_atts)
def btn_select_att(btn_atts):
    global grouping_var
    grouping_var = cb_list.value
btn_atts.on_click(btn_select_att)

In [ ]:
cohort_df = full_df[(full_df['student.cohort.id'].isin(cohort))
                    & (full_df['program.major_id'].isin(programs))
                    & (full_df['campus.region_name'].isin(regions))
                    & (full_df['student.isWoman'].isin(sex))]
assert (set(cohort_df['student.cohort.id'].unique()) == set(cohort)) \
        & (set(cohort_df['program.major_id'].unique()) == set(programs)) \
        & (set(cohort_df['campus.region_name'].unique()) == set(regions)) \
        & (set(cohort_df['student.isWoman'].unique()) == set(sex)) \
                , f'Error! Cohort column includes {existing_cohort} and selected cohort is {cohort}'
print(f'Selected dataframe has {len(cohort_df.index)} rows for:\n cohort(s) {cohort},\n program(s) {programs},\n region(s) {regions},\n and sex {sex}.')

In [ ]:
# Group by student_id and aggregate semesters into a set
grouped = cohort_df.groupby('student.id')['semesters_from.enrollment'].apply(set).reset_index()

# Filter students who have semesters 1, 2, 3, 4, 5, and 6
filtered_students = grouped[grouped['semesters_from.enrollment'].apply(lambda x: {1, 2, 3, 4, 5, 6}.issubset(x))]

# Get the list of student IDs who meet the criteria
valid_student_ids = filtered_students['student.id']

# Filtering rows belonging to the semesters 1, 2, 3, 4, 5, and 6.
filtered_cohort = cohort_df[cohort_df['student.id'].isin(valid_student_ids) 
                            & cohort_df['semesters_from.enrollment'].isin([1, 2, 3, 4, 5, 6])]

len(filtered_cohort['student.id'].unique())

In [ ]:
sem_list = filtered_cohort['semesters_from.enrollment'].unique()
region_list = filtered_cohort['campus.region_name'].unique()
assert set(sem_list) == {1, 2, 3, 4, 5, 6}, f'Error! There is at least one incorrect semester'
print(f'Semesters list {sem_list}')
print(f'Regions: {region_list}')

#### Updating DR region with the last region where the student finalized the studies

In [ ]:
dr_studID_list = list(filtered_cohort[filtered_cohort['campus.region_name'] == 'DR']['student.id'].unique())
len(dr_studID_list)

In [ ]:
for stud_id in dr_studID_list:
    # Get all regions of the student
    campus_list = list(filtered_cohort[filtered_cohort['student.id'] == stud_id]['campus.region_name'].unique())
    # Drop DR region if it exists
    if 'DR' in campus_list:
        campus_list.remove('DR')
    # Assign the last region where the student finalized the studies
    if campus_list:  
        filtered_cohort.loc[filtered_cohort['student.id'] == stud_id, 'campus.region_name'] = campus_list[-1]

In [ ]:
assert len(filtered_cohort['campus.region_name'].unique()) == 4, 'Error number of regions differ from 4' 

##### Computing the Observed_competencies_ratio for students

In [ ]:
# Group by student ID and calculate the ratio of True evaluations
ratio_df = filtered_cohort.groupby(['student.id', 'semesters_from.enrollment'])['subcompetence.level_assigned'].mean().reset_index()
ratio_df.columns = ['student.id', 'semesters_from.enrollment', 'Observed_ratio']

# Merge the ratio back to the original dataframe
filtered_cohort = filtered_cohort.merge(ratio_df, on=['student.id', 'semesters_from.enrollment'])

In [ ]:
tuple_df = ratio_df.groupby(['student.id'])['semesters_from.enrollment'].unique().apply(tuple)
assert tuple_df[tuple_df != (1, 2, 3, 4, 5, 6)].empty, 'Error! There are Students with semesters different to [1, 2, and 3]'

#### Modifying academic program.

 Version 1, all program into 4 knowledge areas.

In [ ]:
# Mapping academic programs to 4 knowledge area
program_mapping = {
    'IIT' : 'IIT',
    'IIS' : 'IIT',
    'IMT' : 'IIT',
    'IC'  : 'IIT',
    'IE'  : 'IIT',
    'IID' : 'IIT',
    'IM'  : 'IIT',
    'IMD' : 'IIT',
    'ICI' : 'ICI',
    'IDM' : 'ICI',
    'INA' : 'ICI',
    'IFI' : 'ICI',
    'ICT' : 'ICT',
    'ITC' : 'ICT',
    'IRS' : 'ICT',
    'ITD' : 'ICT',
    'IBQ' : 'IBQ',
    'IDS' : 'IBQ',
    'IBT' : 'IBQ', 
    'IQ'  : 'IBQ',
    'IAG' : 'IBQ',
    'IAL' : 'IBQ'
}
filtered_cohort['program.major_id'] = filtered_cohort['program.major_id'].replace(program_mapping)
assert len(filtered_cohort['program.major_id'].unique()) == 4, 'Error knowledge areas differ from 4'


Version 2. Transforming entry program data into specific programs.

In [ ]:
# def transform_program_2(group):
#     programs = list(group['program.major_id'].unique())
#     if len(programs) > 1:
#         group['program.major_id'] = programs[1] if programs[0] in ['IIT', 'ICT', 'IBQ', 'ICI'] else programs[0]
#     return group

def transform_program_2(group):
    programs = list(group['program.major_id'].unique())
    if len(programs) > 1:
        selected_program = programs[1] if programs[0] in ['IIT', 'ICT', 'IBQ', 'ICI'] else programs[0]
        group['program.major_id'] = selected_program
    return group

filtered_cohort = filtered_cohort.groupby('student.id', as_index=False).apply(transform_program_2)

##### Filter dataframe

In [ ]:
filtered_cohort = filtered_cohort.drop_duplicates(subset= ['student.id', 'student.nationality', 'student_originSchool.isITESM', 
                                                           'student.cohort.id', 'semesters_from.enrollment','student.isWoman', 
                                                           'campus.region_name', 'program.major_id', 'Observed_ratio'], 
                                                  keep='first')
longitudinal_df = filtered_cohort[['student.id', 'student.nationality', 'student_originSchool.isITESM', 
                                   'student.cohort.id', 'semesters_from.enrollment','student.isWoman', 
                                   'campus.region_name', 'program.major_id', 'Observed_ratio']]
longitudinal_df.set_index(keys=['student.id'], inplace=True)

In [ ]:
assert len(longitudinal_df.index) == len(filtered_cohort['student.id'].unique()) * 6, 'Error! Rows number mismatch student IDs *times 3'
# longitudinal_df.sort_values(by='student.id')

<font color='blue'>**Until here, we have the data filtered by students, but there are multiple rows for each student. Yet, we need to transform it to add Observed competencies ratio as columns for each student, instead of as nwe rows**</font>

In [ ]:
# Pivot the DataFrame
pivot_df = longitudinal_df.pivot(columns='semesters_from.enrollment', values='Observed_ratio')

# Rename the columns to include the semester information
pivot_df.columns = [f'Observed_ratio{col}' for col in pivot_df.columns]

# Reset the index to make 'student.id' a column again
pivot_df = pivot_df.reset_index()

# Add other columns
pivot_df = pivot_df.merge(longitudinal_df[longitudinal_df['semesters_from.enrollment'] == 1], on='student.id').drop(['Observed_ratio',
                                                                                                                     'semesters_from.enrollment'], axis=1)
# Visualize the tidy dataframe
pivot_df

In [ ]:
pivot_df.to_csv(f'EICData\LongitudinalData_Cohort_{cohort}_EntryPrograms_6semesters_4regions.csv')

### 1.2-Statistical-description-of-the-selected-variables

##### Describe numeric variables 

In [ ]:
pivot_df.describe()

##### Describe categorical and boolean variables

In [ ]:
categorical_cols = pivot_df.select_dtypes(include=['object', 'bool', 'category'])
categorical_cols

In [ ]:
for col in categorical_cols:
    print(f"Column: {col}")
    print(f"Number of unique categories: {pivot_df[col].nunique()}")
    print(f"Mode: {pivot_df[col].mode()[0]}")
    print(f"Value counts:\n{pivot_df[col].value_counts()}")
    print("-" * 40)

In [ ]:
print(longitudinal_df.info())

In [ ]:
longitudinal_df.describe()

In [ ]:
print(longitudinal_df.isnull().sum())

### 1.3-Graphical-description-of-the-selected-variables

In [ ]:
for col in categorical_cols:
    fig = px.bar(
        pivot_df[col].value_counts().reset_index(),
        x=col, 
        y='count',
        title=f'Distribution of {col}',
        labels={'index': col, col: 'Count'},
        text_auto=True
    )
    fig.update_layout(
        xaxis_title=col,
        yaxis_title='Count',
        title_x=0.5,  # Center the title
        xaxis_tickangle=-45  # Rotate x-axis labels if needed
    )
    fig.show()

In [ ]:
i = 0
for col in categorical_cols:
    # Prepare the data for the pie chart
    pie_data = pivot_df[col].value_counts().reset_index()
    pie_data.columns = [col, 'count']
    
    # Create the pie chart
    fig = px.pie(
        pie_data, 
        names=col,   # Categorical column for slices
        values='count',  # Numerical values
        #title=f'Distribution of {col}',  # Title of the plot
        labels={col: 'Category', 'count': 'Count'},  # Customize labels
        hole=0,  # Optional: Adds a hole for a donut-style chart
    )
    
    fig.update_traces(
        textposition='inside',  # Place labels inside the slices
        textinfo='label+value+percent',  # Show both percentage and category inside the pie
        showlegend=False,  # Hide external legend
        textfont_size=38,
        marker=dict(
            colors=px.colors.sequential.Greys,  # Use grayscale colors
            line=dict(color='black', width=2)  # Add borders with black color and width of 2
        )
    )
        
    fig.update_layout(
        title_x=0.5,  # Center the title
        title_font_size=38,  # Adjust title font size
        height=600,  # Adjust the overall height of the chart
        width=600,  # Adjust the overall width of the chart
    )

    # Show the figure
    fig.show()
    i+=1

In [ ]:
for col in longitudinal_df.columns:
  longitudinal_df.boxplot(by=col, column='Observed_ratio')
  plt.suptitle('')
  plt.title(f'Boxplots of Observed_ratio by {col}')
  plt.show()